In [1]:
import numpy as np

import pandas as pd

pathToCsvFile = "athlete_events.csv"

# Data exploration

In [2]:
df = pd.read_csv(pathToCsvFile, header=0)
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [3]:
df.describe()

,ID,Age,Height,Weight,Year
count,271116.000000,261642.000000,210945.000000,208241.000000,271116.000000
mean,68248.954396,25.556898,175.338970,70.702393,1978.378480
std,39022.286345,6.393561,10.518462,14.348020,29.877632
min,1.000000,10.000000,127.000000,25.000000,1896.000000
25%,34643.000000,21.000000,168.000000,60.000000,1960.000000
50%,68205.000000,24.000000,175.000000,70.000000,1988.000000
75%,102097.250000,28.000000,183.000000,79.000000,2002.000000
max,135571.000000,97.000000,226.000000,214.000000,2016.000000


# Printing the unique value count for each column

In [4]:
print("Event",len(df.Event.unique()))

print("Name",len(df.Name.unique()))

print("Team",len(df.Team.unique()))

print("Sport",len(df.Sport.unique()))

print("Medal",len(df.Medal.unique()))

print("Games",len(df.Games.unique()))

Event 765
Name 134732
Team 1184
Sport 66
Medal 4
Games 51


In [5]:
def getEntityToIdDict(entityArray):
    entityToId = {}
    i = 1
    for i in range(len(entityArray)):
        entity = entityArray[i]
        if entity not in entityToId:
            entityToId[entity] = i + 1
            i += 1
    return entityToId
    
eventToId = getEntityToIdDict(df.Event.unique())
athleteToId = getEntityToIdDict(df.Name.unique())
teamToID = getEntityToIdDict(df.Team.unique())
sportToID = getEntityToIdDict(df.Sport.unique())
medalToID = getEntityToIdDict(df.Medal.unique())
gamesToID = getEntityToIdDict(df.Games.unique())


# Games table - insert values

In [6]:
games = []
for ind in df.index:
    gameID = gamesToID[df.Games[ind]]
    season = df.Season[ind]
    year = df.Year[ind]
    city = df.City[ind]
    row = (gameID, season , int(year), city)
    games.append(row)
    

 # Events Table - insert values

In [7]:
events = []
for ind in df.index:
    eventID = eventToId[df.Event[ind]]
    eventType = df.Event[ind]
    sportID = sportToID[df.Sport[ind]]
    row = (eventID, eventType, sportID)
    events.append(row)


# Teams Table - insert values

In [8]:
teams = []
for ind in df.index:
    teamID = teamToID[df.Team[ind]]
    country = df.Team[ind]
    noc = df.NOC[ind]
    row = (teamID, country , noc)
    teams.append(row)
    

# Athlets Table - insert values

In [9]:
athletes = []
for ind in df.index:
    athleteID = athleteToId[df.Name[ind]]
    name = df.Name[ind]
    sex = df.Sex[ind]
    age = df.Age[ind]
    height = df.Height[ind]
    weight = df.Weight[ind]

    # Cleaning null data
    if not pd.isna(df.Age[ind]) and  (not pd.isna(df.Name[ind])) and (not pd.isna(df.Sex[ind])) and (not pd.isna(df.Height[ind])) and (not pd.isna(df.Weight[ind])):
        row = (athleteID, name , sex , age, height, weight)
        athletes.append(row)
        

# Sports Table - insert values

In [10]:
sports = []
for ind in df.index:
    sportID = sportToID[df.Sport[ind]]
    sportsType = df.Sport[ind]
    row = (sportID, sportsType)
    sports.append(row)

# Participations - insert values

In [11]:
participations = []
for ind in df.index:
    eventID = eventToId[df.Event[ind]]
    gameID = gamesToID[df.Games[ind]]
    medalType = df.Medal[ind]
    athleteID = athleteToId[df.Name[ind]]
    row = (eventID, gameID, medalType, athleteID)
    participations.append(row)

# Attendance - insert values

In [12]:
attendance = []
for ind in df.index:
    teamID = teamToID[df.Team[ind]]
    gameID = gamesToID[df.Games[ind]]
    athleteID = athleteToId[df.Name[ind]]
    row = (teamID, gameID, athleteID)
    attendance.append(row)

# Database & Table creation

In [13]:
import sqlite3
from sqlite3 import Error

dbPath = "/tmp/olympic-games-sqlite.db"

def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn


def executeQueries(conn, queries):
    cur = conn.cursor()
    for sql in queries:
        print("sql==> ", sql)
        cur.execute(sql)
    conn.commit()

def get_drop_tables_queries():
    teamsSQL = """
    DROP table  IF EXISTS  "Teams";
    """
    
    athletesSQL = """
    DROP table  IF EXISTS  "Athletes";
    """
    
    sportsSQL = """
    DROP table  IF EXISTS  "Sports";
    """
    
    gamesSQL = """
    DROP table  IF EXISTS  "Games";
    """


    eventsSQL = """
    DROP table  IF EXISTS  "Events";
    """
    
    participationsSQL = """
    DROP table  IF EXISTS  "Participations";
    """
    
    attendanceSQL = """
    DROP table  IF EXISTS  "Attendance";
    """
    
    
    allTablesDeleteSql = [teamsSQL, athletesSQL, sportsSQL, gamesSQL, eventsSQL, participationsSQL, attendanceSQL]
    return allTablesDeleteSql;

def get_create_tables_queries():
    teamsSQL = """
    CREATE TABLE IF NOT EXISTS "Teams" (
	"teamID" Integer,
    "country" Text NOT NULL,
    "noc" Text NOT NULL,
    PRIMARY KEY ("teamID")
);
    """;
    
    athletesSQL = """
    CREATE TABLE IF NOT EXISTS "Athletes" (
	"athleteID" Integer,
    "name" Text NOT NULL,
    "sex"	TEXT CHECK("sex" IN ("M", "F")),
    "age" Integer NOT NULL,
    "height" NUMERIC NOT NULL,
    "weight" NUMERIC NOT NULL,
    PRIMARY KEY ("athleteID")
);
    """
    
    
    sportsSQL = """
    CREATE TABLE IF NOT EXISTS "Sports" (
	"sportID" Integer,
    "sportsType" Text NOT NULL,
    PRIMARY KEY ("sportID")
);
    """
    gamesSQL = """
    CREATE TABLE IF NOT EXISTS "Games" (
	"gameID" Integer,
	"season"	TEXT CHECK("season" IN ("Winter", "Summer")),
	"year" Integer NOT NULL,
    "city" Text  NOT NULL,
	PRIMARY KEY("gameID")
);
    """
    
    eventsSQL = """
    CREATE TABLE "Events" (
      "eventID" Integer,
      "eventType" Text NOT NULL,
      "sportID" Integer,
      PRIMARY KEY ("eventID"),
      CONSTRAINT "FK_Events.sportID"
        FOREIGN KEY ("sportID")
          REFERENCES "Sports"("sportID")
);
    """
    
    participationsSQL = """
    CREATE TABLE "Participations" (
      "eventID" Integer,
      "gameID" Integer,
      "medal"	TEXT CHECK("medal" IN ("Gold", "Bronze", "Silver")),
      "athleteID" Integer,
      PRIMARY KEY ("eventID", "gameID", "athleteID"),
      CONSTRAINT "FK_Participations.athleteID"
        FOREIGN KEY ("athleteID")
          REFERENCES "Athletes"("athleteID"),
      CONSTRAINT "FK_Participations.gameID"
        FOREIGN KEY ("gameID")
          REFERENCES "Games"("gameID"),
      CONSTRAINT "FK_Participations.eventID"
        FOREIGN KEY ("eventID")
          REFERENCES "Events"("eventID") 
);
    """
    
    attendanceSQL = """
    CREATE TABLE "Attendance" (
      "teamID" Integer,
      "gameID" Integer,
      "athleteID" Integer,
      PRIMARY KEY ("teamID", "gameID", "athleteID"),
      CONSTRAINT "FK_Attendance.teamID"
        FOREIGN KEY ("teamID")
          REFERENCES "Teams"("teamID"),
      CONSTRAINT "FK_Attendance.athleteID"
        FOREIGN KEY ("athleteID")
          REFERENCES "Athletes"("athleteID"),
      CONSTRAINT "FK_Attendance.gameID"
        FOREIGN KEY ("gameID")
          REFERENCES "Teams"("gameID")
);
    """
    
    
    allTablesCreateSql = [teamsSQL, athletesSQL, sportsSQL, gamesSQL, eventsSQL, participationsSQL, attendanceSQL]
    return allTablesCreateSql

def insert_entities(conn,sql, entities):
    cur = conn.cursor()
    cur.executemany(sql, entities)
    conn.commit()
    return cur.lastrowid

def main():
    # create a database connection
    conn = create_connection(dbPath)
    with conn:
        # drop all tables 1st
        executeQueries(conn, get_drop_tables_queries())
        
        
        # create all tables
        executeQueries(conn, get_create_tables_queries())
        
        # insert teams data
        tableName = "Teams"
        teamsSql = "INSERT OR REPLACE INTO {}(teamID, country, noc) values(?,?,?)".format(tableName)
        teamsCount = insert_entities(conn, teamsSql, teams)
        
        # insert Athletes data
        tableName = "Athletes"
        athletesSql = "INSERT OR REPLACE INTO {}(athleteID, name, sex, age, height, weight) values(?, ?, ?, ?, ?, ?)".format(tableName)
        athletesCount = insert_entities(conn, athletesSql, athletes)
        
         # insert Sports data
        tableName = "Sports"
        sportsSQL = "INSERT OR REPLACE INTO {}(sportID, sportsType) values(?,?)".format(tableName)
        sportsCount = insert_entities(conn, sportsSQL, sports)
        
         # insert Games data
        tableName = "Games"
        gamesSQL = "INSERT OR REPLACE INTO {}(gameID, season, year, city) values(?, ?, ?, ?)".format(tableName)
        gamesCount = insert_entities(conn, gamesSQL, games)
        
        # insert Events data
        tableName = "Events"
        eventsSQL = "INSERT OR REPLACE INTO {}(eventID, eventType, sportID) values(?, ?, ?)".format(tableName)
        eventsCount = insert_entities(conn, eventsSQL, events)
        
        # insert Participations data
        tableName = "Participations"
        participationsSQL = "INSERT OR REPLACE INTO {}(eventID, gameID, medal, athleteID) values(?, ?, ?, ?)".format(tableName)
        participationsCount = insert_entities(conn, participationsSQL, participations)

        # insert Attendance data
        tableName = "Attendance"
        attendanceSQL = "INSERT OR REPLACE INTO {}(teamID, gameID, athleteID) values(?, ?, ?)".format(tableName)
        attendanceCount = insert_entities(conn, attendanceSQL, attendance)
       

if __name__ == '__main__':
    main()

sql==>  
    DROP table  IF EXISTS  "Teams";
    
sql==>  
    DROP table  IF EXISTS  "Athletes";
    
sql==>  
    DROP table  IF EXISTS  "Sports";
    
sql==>  
    DROP table  IF EXISTS  "Games";
    
sql==>  
    DROP table  IF EXISTS  "Events";
    
sql==>  
    DROP table  IF EXISTS  "Participations";
    
sql==>  
    DROP table  IF EXISTS  "Attendance";
    
sql==>  
    CREATE TABLE IF NOT EXISTS "Teams" (
	"teamID" Integer,
    "country" Text NOT NULL,
    "noc" Text NOT NULL,
    PRIMARY KEY ("teamID")
);
    
sql==>  
    CREATE TABLE IF NOT EXISTS "Athletes" (
	"athleteID" Integer,
    "name" Text NOT NULL,
    "sex"	TEXT CHECK("sex" IN ("M", "F")),
    "age" Integer NOT NULL,
    "height" NUMERIC NOT NULL,
    "weight" NUMERIC NOT NULL,
    PRIMARY KEY ("athleteID")
);
    
sql==>  
    CREATE TABLE IF NOT EXISTS "Sports" (
	"sportID" Integer,
    "sportsType" Text NOT NULL,
    PRIMARY KEY ("sportID")
);
    
sql==>  
    CREATE TABLE IF NOT EXISTS "Games" (
	"gameID" In